In [1]:
import numpy as np
import tensorflow 
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers 

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D 
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [2]:
tensorflow.__version__

'2.3.0'

In [3]:
tensorflow.keras.__version__

'2.4.0'

# Image Data Agumentation

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

# Loading our data and performing data agumentation

In [5]:

x_train = train_datagen.flow_from_directory(r"C:\Users\Siri\Desktop\dataset\train_set",target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode="categorical")

x_test = test_datagen.flow_from_directory(r"C:\Users\Siri\Desktop\dataset\test_set",target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode="categorical")

Found 742 images belonging to 4 classes.
Found 198 images belonging to 4 classes.


In [6]:
print(x_train.class_indices)

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [7]:
print(x_test.class_indices)

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [8]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 220, 1: 156, 2: 198, 3: 168})

# creating the model


In [9]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=4, activation='softmax')) # softmax for more than 2


In [10]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 5

# compiling the model

In [39]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# fitting the model

In [40]:
 classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))

Epoch 1/20
149/149 [==============================] - 48s 321ms/step - loss: 1.2289 - accuracy: 0.4340 - val_loss: 1.0448 - val_accuracy: 0.5909
Epoch 2/20
149/149 [==============================] - 58s 390ms/step - loss: 0.9660 - accuracy: 0.6051 - val_loss: 0.7845 - val_accuracy: 0.7525
Epoch 3/20
149/149 [==============================] - 61s 412ms/step - loss: 0.7496 - accuracy: 0.7049 - val_loss: 0.6672 - val_accuracy: 0.7525
Epoch 4/20
149/149 [==============================] - 57s 380ms/step - loss: 0.6355 - accuracy: 0.7642 - val_loss: 0.6264 - val_accuracy: 0.8030
Epoch 5/20
149/149 [==============================] - 64s 429ms/step - loss: 0.5772 - accuracy: 0.7749 - val_loss: 0.5939 - val_accuracy: 0.8131
Epoch 6/20
149/149 [==============================] - 63s 424ms/step - loss: 0.5025 - accuracy: 0.8127 - val_loss: 0.6985 - val_accuracy: 0.7828
Epoch 7/20
149/149 [==============================] - 61s 409ms/step - loss: 0.4491 - accuracy: 0.8342 - val_loss: 0.5495 - val_ac

# saving our model

In [41]:
classifier.save('disaster.h5')

In [42]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

# Predicting our results

In [43]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("disaster.h5")

In [44]:
img = image.load_img(r"D:\geethanjali externship\earthquake.jpg",grayscale=False,
                     target_size= (64,64))
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
pred = model.predict_classes(x)
pred

array([0], dtype=int64)

In [45]:
index=['Cyclone','Earthquake','Flood','Wildfire']
result=str(index[pred[0]])
result

'Cyclone'